<a href="https://colab.research.google.com/github/dariocostta/Metodos-Numericos-I/blob/master/Unid-3-Autovetores-e-Autovalores/Tarefa_12_MN2_Transf_Similaridade_HouseHolder_e_QR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Métodos Numéricos II (2023.1 - T02)
- Aluno: Dario Filipe da Silva Costa
- Matrícula: 422156
- Email: dariocosta@alu.ufc.br

In [79]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
from IPython.display import display, Latex

# Métodos anteriores: 

## Tarefa 10: Potência Regular

In [80]:
def potReg(A:np.ndarray, v0:np.ndarray, e:float=1e-7,
           iterMax=1e5) -> tuple[np.ndarray, float]:
  '''
  Supõe v0 um vetor coluna e A uma matriz.

  PARAMETROS:
  - - - - -
  A  : ndarray - Matriz do problema
  v0 : ndarray - Chute inicial do autovetor x_1 (vetor coluna)
  '''
  v=v0.copy()
  lamb = 0

  iter = 0 #Contador de iterações
  while iter<=iterMax:
    v_old=v.copy()
    lamb_old = lamb
    v_old=(v_old)/((v_old.T@v_old)**0.5)
    v = A@v_old
    lamb = v_old.T@v
    iter+=1

    if abs((lamb - lamb_old)/lamb) < e: #Teste de parada
      break

  if iter > iterMax:
    print("As iterações superaram o máximo estipulado, chance de não ter convergido.")

  return v/v[-1], lamb

## Tarefa 11: Potência Inversa e Deslocada


In [81]:
def decompLU(A: np.ndarray)-> tuple[np.ndarray, np.ndarray]:
  '''
  Recebe A e devolve a decomposição em LU=A
  L= Lower Triag. e U = Upper Triag.
  '''
  n = len(A)
  L = np.zeros((n, n))
  U = np.zeros((n, n))
  for c in range(n):
    for l in range(c+1):
      sr = sum(L[l][k]*U[k][c] for k in range(l)) if l>0 else 0
      U[l][c] = A[l][c] - sr

    for l in range(c, n):
      sr=sum(L[l][k]*U[k][c] for k in range(c)) if c>0 else 0
      L[l][c] = (A[l][c]-sr)/U[c][c]

  return L, U

def solverLU(L: np.ndarray, U:np.ndarray, b:np.ndarray)->np.ndarray:
  ''' Devolve x que é solução do sistema LUx=b
  '''
  n = len(L)
  x = np.zeros(n)
  y = np.zeros(n)
  for i in range(n):
    r = sum(L[i][k]*y[k] for k in range(i)) if i>0 else 0
    y[i] = (b[i]-r)/L[i][i]
  for i in range(n-1, -1, -1):
    r = sum(U[i][k]*x[k] for k in range(i+1, n)) if i<n-1 else 0
    x[i] = (y[i]-r)/U[i][i]
  return x

def potInv(A:np.ndarray, v0:np.ndarray, e:float=1e-7,
           iterMax=1e5) -> tuple[np.ndarray, float]:
  '''
  Supõe v0 um vetor coluna e A uma matriz.

  PARAMETROS:
  - - - - -
  A  : ndarray - Matriz do problema
  v0 : ndarray - Chute inicial do autovetor x_n (vetor coluna)
  '''
  v=v0.copy()
  lamb = 0
  # Obtendo a L e U <----
  L, U = decompLU(A)

  iter = 0 #Contador de iterações
  while iter<=iterMax:
    v_old=v.copy()
    lamb_old = lamb
    v_old=(v_old)/((v_old.T@v_old)**0.5)
    v = solverLU(L, U, v_old) #<-- Obtendo A^-1v_old=v_new pela
                              # solucao do sistema v_old=Av_new
    lamb = v_old.T@v

    iter+=1
    if abs((lamb - lamb_old)/lamb) < e: #Teste de parada
      break   
  
  if iter > iterMax:
    print("As iterações superaram o máximo estipulado, chance de não ter convergido.")

  return v/v[-1], 1/lamb

## Deslocada

In [82]:
def potDesloc(A:np.ndarray, v0:np.ndarray, u:float, e=1e-7)->tuple[np.ndarray, float]:
  I=np.identity(len(A))
  Abar = A-u*I
  v, lamb = potInv(Abar, v0, e)
  return v, lamb+u

## tryAll()

Essa função calcula os dois extremos autovalores e autovetores usando as `potReg` e `potInv` e então usa a `potDesloc` para encontrar os internos.
Ela discretiza dentro dos intervalos entre cada autovalor já encontrado, tentativas de $\mu$.

In [83]:
def tryAll(A:np.ndarray, e=1e-7, iterMax=1e2)->dict:
  n=len(A)
  resp={}
  v = np.ones(n)

  x1, lamb1=potReg(A, v, e)
  xn, lambn=potInv(A, v, e)

  resp[lambn]=[xn, lambn]
  resp[lamb1]=[x1, lamb1]
  
  k=2
  iter = 2 # comeca em 2 pois vou usar no numero de divisões do intervalo
  while iter<iterMax and len(resp)<n:
    ks = list(sorted(resp.keys()))

    for i in range(len(ks)-1):
      #dividir o intervalo entre os autovalores já encontrados em iter partes
      us=[ks[i]+((ks[i+1]-ks[i])/iter)*k for k in range(1, iter)]

      for j in range(len(us)):
        xnew, lambnew = potDesloc(A, v, us[j], e)
      
        new = True
        for test in resp.keys():
          if abs(lambnew-test) < e: #Testando se ele já esta na lista
            new = False
        if new:
          resp[lambnew]=[xnew, lambnew] #adiconando o novo encontrado

  keys = list(sorted(resp.keys(), key=abs, reverse=True)) #organizando
  r = {k: resp[k]  for k in keys}
  return r

# Tarefa 12: Householder e QR

Obs: Comecei a exibir os autovetores na forma $\vec{x} = (a_1/a_n, ~a_2/a_n ,\cdots, ~1)$

pois é assim que é exibido no Wolfram e outros sites que visitei. E isso não muda a direção, apenas dividi pelo escalar da ultima posição, $a_n$.

###Testando para as matrizes:

- $A_1 = \left[\begin{array}{c c c}
5 & 2 & 1\\
2 & 3 & 1\\
1 & 1 & 2
\end{array}\right]$

- $A_2 = \left[\begin{array}{r r r}
-14 & 1 & -2\\
1 & -1 & 1\\
-2 & 1 & -11
\end{array}\right]$

- $A_3 = \left[\begin{array}{r r r r r}
40 & 8 & 4 & 2 & 1 \\
8 & 30 & 12 & 6 & 2\\
4 & 12 & 20 & 1 & 2\\
2 & 6 & 1 & 25 & 4\\
1 & 2 & 2 & 4 & 5
\end{array}\right]$

In [84]:
rnd = lambda x: round(x, 5)
As = {1:np.array([
       [5, 2, 1],
       [2, 3, 1], 
       [1, 1, 2]
       ]),
     2: np.array([
       [-14, 1, -2],
       [1, -1,   1], 
       [-2, 1, -11]
       ]),
     3:np.array([
       [40, 8, 4, 2, 1],
       [8, 30, 12, 6, 2],
       [4, 12, 20, 1, 2],
       [2, 6, 1, 25, 4],
       [1, 2, 2, 4, 5]
      ])
     }

## HouseHolder

In [85]:
def constHH(Aold : np.ndarray, j : int, m : int)->np.ndarray:
  v = np.zeros(m)
  vl = np.zeros(m)
  for k in range(j+1, m):
    v[k]=Aold[k][j]
  Lv = (v.T@v)**(1/2) # ||v||_2
  vl[j+1]=Lv
  N=v-vl
  n=N/((N.T@N)**(1/2)) # ||N||_2
  Hj=np.identity(m)-2*np.outer(n, n.T) # Produto externo
  return Hj

def metodoHH(A:np.ndarray, e=1e-10)->tuple[np.ndarray, np.ndarray]:
  n, m = len(A), len(A[0])
  T = A.copy() # "A_old"
  H = np.identity(n)
  for j in range(m-2):
    Hj = constHH(T, j, m)
    Anew = Hj@T@Hj
    T = Anew.copy()
    H = H@Hj
  
  # e é o limite que o valor será considerado zero
  # (pois sobra alguns restos de arredondamento na T)
  for i in range(n):
    for j in range(m):
        if abs(T[i][j]) < e:
          T[i][j]=0

  return T, H

In [86]:
rnd = lambda x: round(x, 5)
A = As[3]

T, H = metodoHH(A)
display(Latex(r'\bar{A} = '))
print(tabulate(T, tablefmt='fancy_grid'))
print('H = ')
print(tabulate(H, tablefmt='fancy_grid'))


r = tryAll(T)
l='$'
for i, (x, lamb) in enumerate(r.values()):
  x=list(map(rnd, x))
  lamb=rnd(lamb)
  l += f'\lambda_{i+1}={lamb}, ~~'
  display(Latex(r'\bar{x}_'+f'{i+1}={x}'))
l+='$'
display(Latex(l))
print('Como podemos observar, os autovalores são iguais ao problema original,'+
      '\nmas os autovetores não, para obterlos vamos pré-multiplicar por H.')
for i, (x, lamb) in enumerate(r.values()):
  x=H@x
  x=list(map(rnd, x/x[-1]))
  display(Latex(r'H\bar{x}_'+f'{i+1}={x} ='+r'x_'+f'{i+1}'))

print('\nCalculando direto no problema original:')
r = tryAll(A)
print(tabulate(A, tablefmt='grid'))
l='$'
for i, (x, lamb) in enumerate(r.values()):
  x=list(map(rnd, x))
  lamb=rnd(lamb)
  l += f'\lambda_{i+1}={lamb}, ~~'
  display(Latex(f'x_{i+1}={x}'))
l+='$'
display(Latex(l))

<IPython.core.display.Latex object>

╒══════════╤══════════╤══════════╤══════════╤═════════╕
│ 40       │  9.21954 │  0       │  0       │ 0       │
├──────────┼──────────┼──────────┼──────────┼─────────┤
│  9.21954 │ 39.8235  │  4.06759 │  0       │ 0       │
├──────────┼──────────┼──────────┼──────────┼─────────┤
│  0       │  4.06759 │ 15.0687  │  6.29137 │ 0       │
├──────────┼──────────┼──────────┼──────────┼─────────┤
│  0       │  0       │  6.29137 │ 17.59    │ 5.86698 │
├──────────┼──────────┼──────────┼──────────┼─────────┤
│  0       │  0       │  0       │  5.86698 │ 7.51779 │
╘══════════╧══════════╧══════════╧══════════╧═════════╛
H = 
╒═══╤══════════╤════════════╤════════════╤═══════════╕
│ 1 │ 0        │  0         │  0         │  0        │
├───┼──────────┼────────────┼────────────┼───────────┤
│ 0 │ 0.867722 │ -0.442337  │  0.0885626 │  0.208695 │
├───┼──────────┼────────────┼────────────┼───────────┤
│ 0 │ 0.433861 │  0.552137  │ -0.603674  │ -0.377475 │
├───┼──────────┼────────────┼────────────┼───────

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Apos pré-multiplicarmos os $\bar{x}_i$ pela $H$, obtemos os $x_i$ da matriz original.

## QR

In [87]:
def constQ(a_jj : float, a_ij : float, j : int, m : int)->np.ndarray:
  Qj = np.identity(m)
  Qj[j][j]=a_jj/np.sqrt(a_ij**2 + a_jj**2) #cos(theta)
  Qj[j][j+1]=-a_ij/np.sqrt(a_ij**2 + a_jj**2) #-sen(theta)

  Qj[j+1][j]=a_ij/np.sqrt(a_ij**2 + a_jj**2) #sen(theta)
  Qj[j+1][j+1]=a_jj/np.sqrt(a_ij**2 + a_jj**2) #cos(theta)

  return Qj #Qj.T é Matriz de rotação para zerar 1 elemento abaixo da diagonal na coluna j.

def decompQR(A : np.ndarray, m: int) -> tuple[np.ndarray, np.ndarray]: #->(Q, R)
  R, Q = A.copy(), np.identity(m)
  for j in range(m-1):
    Qj = constQ(R[j][j] , R[j+1][j], j, m)
    R = Qj.T@R
    Q = Q@Qj
  return Q, R

def metodoQR(T: np.ndarray, H:np.ndarray, e = 1e-6) -> tuple[np.ndarray, np.ndarray]: #-> (D, P)
  lowerD = lambda A: sum((A[i][j])**2 for i in range(1, len(A)) for j in range(0, i)) #Soma dos Elementos abaixo da diagonal
  D, P = T.copy(), H.copy()
  val =  lowerD(D)
  n, m = len(D), len(D[0])
  while val > e:
    Q, R = decompQR(D, m)
    D = R@Q
    P = P@Q
    val = lowerD(D)
  lamb = np.array([D[i][i] for i in range(m)])

  #Deixando os autovetores na mesma forma que o wolfram deixa
  for i in range(n): 
    for j in range(m):
      P[i][j]=P[i][j]/P[n-1][j]
                    
  return P, lamb

In [88]:
for i in As.keys():
  A = As[i]
  T, H = metodoHH(A)
  P, lamb = metodoQR(T, H)
  print('Matriz A:')
  print(tabulate(A, tablefmt='fancy_grid'))
  print('Matriz T(depois do HouseHolder):')
  print(tabulate(T, tablefmt='fancy_grid'))
  print('Matriz dos autovetores P(depois do QR):')
  print(tabulate(P, tablefmt='fancy_grid'))
  print('Vetor dos autovalores:')
  print(list(lamb))
  print(10*'-----//-----')

Matriz A:
╒═══╤═══╤═══╕
│ 5 │ 2 │ 1 │
├───┼───┼───┤
│ 2 │ 3 │ 1 │
├───┼───┼───┤
│ 1 │ 1 │ 2 │
╘═══╧═══╧═══╛
Matriz T(depois do HouseHolder):
╒═════════╤══════════╤══════╕
│ 5       │  2.23607 │  0   │
├─────────┼──────────┼──────┤
│ 2.23607 │  3.6     │ -0.2 │
├─────────┼──────────┼──────┤
│ 0       │ -0.2     │  1.4 │
╘═════════╧══════════╧══════╛
Matriz dos autovetores P(depois do QR):
╒═════════╤══════════╤═══════════╕
│ 2.82288 │ -1.00178 │  0.176114 │
├─────────┼──────────┼───────────┤
│ 1.82288 │  1.00275 │ -0.821311 │
├─────────┼──────────┼───────────┤
│ 1       │  1       │  1        │
╘═════════╧══════════╧═══════════╛
Vetor dos autovalores:
[6.64575131106458, 1.9999991626822762, 1.3542495262531382]
-----//----------//----------//----------//----------//----------//----------//----------//----------//----------//-----
Matriz A:
╒═════╤════╤═════╕
│ -14 │  1 │  -2 │
├─────┼────┼─────┤
│   1 │ -1 │   1 │
├─────┼────┼─────┤
│  -2 │  1 │ -11 │
╘═════╧════╧═════╛
Matriz T(depois do